In [0]:
from delta.tables import * 

In [0]:
%sql
select * from analytics_star_schema.deltashare.cust_stream 
order by customer_id desc limit 5

In [0]:
%sql
select * from analytics_star_schema.deltashare.customers
order by customer_id desc limit 5

In [0]:
%sql
select * from databricks_simulated_retail_customer_data.v01.customers
order by customer_id desc limit 5

## before count

In [0]:
df_customers = spark.read.table('analytics_star_schema.deltashare.customers')

df_customers.count()

In [0]:
df_customers_streams = spark.read.table('analytics_star_schema.deltashare.cust_stream')
df_customers_streams.count()

In [0]:
customers_target = DeltaTable.forName(spark, "analytics_star_schema.deltashare.customers")


customer_source = spark.readStream.table('analytics_star_schema.deltashare.cust_stream')

"""customers_target.alias('t').merge(customer_source.alias('s'), 't.customer_id = s.customer_id').whenMatchedUpdateAll()\
             .whenNotMatchedInsertAll()\
             .execute()"""

In [0]:
(
  spark.readStream
    .table("analytics_star_schema.deltashare.cust_stream")
    .writeStream
    .format("delta")
    .trigger(once=True)
    .option("checkpointLocation", "/Volumes/analytics_star_schema/deltashare/checkpointing/customers_append")
    .toTable("analytics_star_schema.deltashare.customers_stream_append")
)

In [0]:
customers_target.alias("t").merge(
    customer_source.alias("s"),
    """
    LOWER(t.customer_name) = LOWER(s.name)
    AND LOWER(t.city) = LOWER(s.city)
    AND LOWER(t.state) = LOWER(s.state)
    """
).whenNotMatchedInsert(values={
    "customer_id": "s.customer_id",
    "customer_name": "s.name",
    "state": "s.state",
    "city": "s.city",
    "postcode": "s.zip_code",
    "street": "s.address",
    "valid_from": "s.timestamp",
    "valid_to": "NULL"
}).execute()

In [0]:
(
  customer_source
    .writeStream
    .format("delta")
    .trigger(once=True)
    .option("checkpointLocation", "/Volumes/analytics_star_schema/deltashare/checkpointing/customers_t")
    .toTable("analytics_star_schema.deltashare.customers")
)